# Plan to Medal: INGV Volcanic Eruption Prediction

Objectives:
- Build a fast, reliable baseline with robust CV and no leakage.
- Use GPU-accelerated models (XGBoost/CatBoost) and iterate quickly.
- Engineer strong timeseries summary features; validate; then improve via error analysis and ensembling.

Workflow:
1) Environment & GPU
- Verify GPU availability; prefer XGBoost/CatBoost (LightGBM here is CPU-only).

2) Data audit & schema
- Inspect train.csv (targets, IDs), sample_submission.csv, and the per-segment CSVs in train/ and test/.
- Confirm target: time_to_eruption (seconds).
- Determine grouping/stratification keys (e.g., volcano id if present, or file-level CV).

3) Validation protocol
- Lock a deterministic KFold/GroupKFold that mirrors test conditions.
- Likely GroupKFold by volcano or station/source if available; else KFold with multiple seeds.
- Prevent leakage: fit scalers/feature selection inside folds only.

4) Fast baseline features (FE v1)
- Compute per-file statistical summaries across channels: mean, std, min, max, ptp, quantiles (e.g., q1, q5, q10, q25, q50, q75, q90, q95, q99), skew, kurtosis.
- Rolling/windowed stats over multiple window sizes (if cheap): std/mean of windows; maybe only 1–2 windows for baseline.
- Frequency-domain: simple FFT energy bands, spectral centroid, dominant freq, spectral entropy.
- Counts: zero-crossing rate, RMS, number of peaks (lightweight).
- Log-safe transforms where needed.
- Cache features to parquet/npy for reuse.

5) Modeling baseline
- Train XGBoost regressor with GPU, early stopping; log per-fold metrics and time.
- Evaluate MAE OOF; save OOF preds and feature importances.

6) Improvement loop
- Feature ablations; add richer spectral features (Welch PSD bands), autocorr features, trend features.
- Try CatBoost with GPU; blend with XGB by OOF-calibrated weighted average.
- Tune key params (learning_rate, depth, n_estimators) narrowly based on OOF.

7) Error analysis
- Slice OOF errors by file/volcano/station, by feature bins; fix top error buckets.
- Check adversarial validation if domain shift suspected.

8) Submission
- Generate test features using the exact pipeline; create submission.csv.
- Sanity checks: head/tail, shape equals sample_submission, MAE proxy if any CV on pseudo folds.

Logging & Efficiency:
- Always print progress, fold indices, timings.
- Start with subsample runs (e.g., 200–500 files, 2 folds) to validate pipeline before full run.
- Cache intermediate artifacts; avoid recomputing features.

Next Actions:
- Verify GPU and inspect train.csv/sample_submission schema.
- Draft a fast feature-extraction function and run on a small subset to validate alignment with train.csv.
- Establish CV and train a first XGBoost baseline.

In [1]:
# Environment check + schema inspection
import os, sys, subprocess, json, glob, time
import pandas as pd
import numpy as np

print('Python version:', sys.version)
print('CWD:', os.getcwd())

# GPU check via nvidia-smi (lightweight)
def gpu_info():
    try:
        out = subprocess.check_output(['nvidia-smi', '--query-gpu=name,memory.total', '--format=csv,noheader'], stderr=subprocess.STDOUT).decode().strip()
        print('nvidia-smi:', out)
    except Exception as e:
        print('nvidia-smi not available or failed:', e)

gpu_info()

# Check xgboost availability/version
try:
    import xgboost as xgb
    print('xgboost version:', xgb.__version__)
except Exception as e:
    print('xgboost import failed:', e)

train_csv_path = 'train.csv'
test_dir = 'test'
train_dir = 'train'
ss_path = 'sample_submission.csv'

assert os.path.exists(train_csv_path), 'train.csv missing'
assert os.path.exists(ss_path), 'sample_submission.csv missing'
assert os.path.isdir(train_dir), 'train/ dir missing'
assert os.path.isdir(test_dir), 'test/ dir missing'

train_df = pd.read_csv(train_csv_path)
ss_df = pd.read_csv(ss_path)
print('train.csv shape:', train_df.shape)
print('train.csv columns:', list(train_df.columns))
print(train_df.head(3))
print('sample_submission shape:', ss_df.shape)
print('sample_submission columns:', list(ss_df.columns))
print(ss_df.head(3))

# Infer ID columns
id_col_candidates = [c for c in train_df.columns if 'id' in c.lower() or 'segment' in c.lower()]
target_candidates = [c for c in train_df.columns if 'time' in c.lower() and 'eruption' in c.lower()]
print('ID candidates:', id_col_candidates)
print('Target candidates:', target_candidates)

# List a few files in train/ and test/
train_files = sorted(glob.glob(os.path.join(train_dir, '*.csv')))
test_files = sorted(glob.glob(os.path.join(test_dir, '*.csv')))
print('n_train_files:', len(train_files), 'n_test_files:', len(test_files))
print('train sample files:', [os.path.basename(p) for p in train_files[:5]])
print('test sample files:', [os.path.basename(p) for p in test_files[:5]])

# Try to read one train CSV to inspect sensors and length
sample_path = train_files[0] if train_files else None
if sample_path:
    sdf_head = pd.read_csv(sample_path, nrows=5)
    print('Sample train file:', os.path.basename(sample_path))
    print('Sample file head shape:', sdf_head.shape)
    print('Sample file columns:', list(sdf_head.columns))
    # Try reading only header to infer dtypes quickly
    sdf = pd.read_csv(sample_path)
    print('Sample full file shape:', sdf.shape)
    # Basic stats
    numeric_cols = [c for c in sdf.columns if sdf[c].dtype != 'O']
    print('Numeric cols:', len(numeric_cols))
    # If a time column exists, inspect
    time_like = [c for c in sdf.columns if 'time' in c.lower()]
    if time_like:
        print('Time-like columns in segment:', time_like[:3])
    # Estimate sampling rate if time present
    if time_like:
        tcol = time_like[0]
        t = sdf[tcol].values
        if len(t) > 1:
            dt = np.diff(t[:1000])
            dt = dt[np.isfinite(dt)]
            if dt.size:
                fs_est = 1.0 / np.median(dt)
                print('Estimated sampling rate (Hz):', fs_est)
else:
    print('No train files found!')

# Cross-check that IDs from train.csv map to files
if id_col_candidates:
    id_col = id_col_candidates[0]
    train_ids = set(str(x) + '.csv' for x in train_df[id_col].astype(str).values)
    files_set = set(os.path.basename(p) for p in train_files)
    missing = list(train_ids - files_set)
    print('IDs in train.csv not found as files:', len(missing))
    print('First few missing IDs:', missing[:5])

print('Inspection complete.')

Python version: 3.11.0rc1 (main, Aug 12 2022, 10:02:14) [GCC 11.2.0]
CWD: /app/agent_run_states/predict-volcanic-eruptions-ingv-oe-spray-20250913-034310


nvidia-smi: Tesla T4, 15360 MiB


xgboost version: 2.1.4
train.csv shape: (3987, 2)
train.csv columns: ['segment_id', 'time_to_eruption']
   segment_id  time_to_eruption
0  1410693848          20109998
1   987159268           7202883
2  1990984540          28138930
sample_submission shape: (444, 2)
sample_submission columns: ['segment_id', 'time_to_eruption']
   segment_id  time_to_eruption
0   951290289                 0
1   508758258                 0
2  1566132188                 0
ID candidates: ['segment_id']
Target candidates: ['time_to_eruption']
n_train_files: 3987 n_test_files: 444
train sample files: ['1000015382.csv', '1000554676.csv', '1000745424.csv', '1001461087.csv', '1001732002.csv']
test sample files: ['1003520023.csv', '1004346803.csv', '1007996426.csv', '1009749143.csv', '1016956864.csv']
Sample train file: 1000015382.csv
Sample file head shape: (5, 10)
Sample file columns: ['sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5', 'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10']
Sample

In [2]:
# Feature Extraction v1: stats + simple spectral bands; subsample build
import os, glob, time, math
import numpy as np
import pandas as pd
from tqdm import tqdm

np.seterr(all='ignore')

train_df = pd.read_csv('train.csv')
train_path = 'train'
test_path = 'test'

def zero_crossings(x):
    x = np.asarray(x, dtype=np.float32)
    return ((x[:-1] * x[1:]) < 0).sum()

def hjorth_params(x):
    # activity, mobility, complexity
    x = np.asarray(x, dtype=np.float32)
    if x.size < 3:
        return (np.nan, np.nan, np.nan)
    dx = np.diff(x)
    ddx = np.diff(dx)
    var_x = np.var(x)
    var_dx = np.var(dx)
    var_ddx = np.var(ddx) if ddx.size else np.nan
    activity = var_x
    mobility = math.sqrt(var_dx / var_x) if var_x > 0 else np.nan
    complexity = math.sqrt(var_ddx / var_dx) / mobility if (var_dx > 0 and mobility not in (0, np.nan)) else np.nan
    return activity, mobility, complexity

def linear_trend_slope(x):
    x = np.asarray(x, dtype=np.float32)
    n = x.size
    if n < 2:
        return np.nan
    idx = np.arange(n, dtype=np.float32)
    # closed form slope of least squares fit
    idx_mean = (n - 1) / 2.0
    x_mean = x.mean() if n else 0.0
    cov = ((idx - idx_mean) * (x - x_mean)).sum()
    var = ((idx - idx_mean) ** 2).sum()
    return cov / var if var > 0 else np.nan

def spectral_features(x, n_bands=6, downsample=10):
    # simple rfft-based band energies, centroid, entropy, flatness
    x = np.asarray(x, dtype=np.float32)
    if downsample > 1:
        x = x[::downsample]
    x = x - np.nanmean(x)
    x = np.nan_to_num(x)
    n = x.size
    if n < 8:
        return {}, []
    # next power of 2 for speed
    nfft = 1 << (n - 1).bit_length()
    X = np.fft.rfft(x, n=nfft)
    P = (np.abs(X) ** 2).astype(np.float32)
    # ignore DC
    P[0] = 0.0
    # define bands over frequency bins (1..len(P)-1) quasi-log
    L = len(P)
    edges = np.unique(np.round(np.geomspace(1, L - 1, num=n_bands + 1)).astype(int))
    if edges[-1] != L - 1:
        edges[-1] = L - 1
    band_energies = []
    feats = {}
    for i in range(len(edges) - 1):
        a, b = edges[i], edges[i + 1]
        if b <= a:
            band_energies.append(0.0)
            feats[f'band_{i}_energy'] = 0.0
        else:
            e = float(P[a:b].sum())
            band_energies.append(e)
            feats[f'band_{i}_energy'] = math.log1p(e)
    total = float(np.sum(P)) + 1e-12
    # spectral centroid (bin index weighted)
    idx = np.arange(L, dtype=np.float32)
    centroid = float((idx * P).sum() / total)
    feats['spec_centroid'] = centroid
    # spectral entropy
    p = P / total
    p = p[p > 0]
    spec_entropy = -float(np.sum(p * np.log(p)))
    feats['spec_entropy'] = spec_entropy
    # spectral flatness
    geo_mean = float(np.exp(np.mean(np.log(P[P > 0])))) if np.any(P > 0) else 0.0
    arith_mean = float(np.mean(P)) + 1e-12
    feats['spec_flatness'] = geo_mean / arith_mean
    # dominant frequency bin and power
    dom_idx = int(np.argmax(P))
    feats['dom_freq_bin'] = dom_idx
    feats['dom_power_log1p'] = math.log1p(float(P[dom_idx]))
    return feats, band_energies

def extract_features_for_file(path):
    try:
        df = pd.read_csv(path)
        arr = df.values.astype(np.float32)  # shape (T, C)
        T, C = arr.shape
        feats = {'rows': T, 'cols': C, 'missing_frac': float(np.isnan(arr).mean())}
        # per-sensor features
        for j in range(C):
            x = arr[:, j]
            x = x.astype(np.float32)
            # basic stats
            feats[f's{j}_mean'] = float(np.nanmean(x))
            feats[f's{j}_std'] = float(np.nanstd(x))
            feats[f's{j}_min'] = float(np.nanmin(x))
            feats[f's{j}_max'] = float(np.nanmax(x))
            feats[f's{j}_ptp'] = float(np.nanmax(x) - np.nanmin(x))
            qs = [1,5,10,25,50,75,90,95,99]
            qv = np.nanpercentile(x, qs)
            for qi, q in enumerate(qs):
                feats[f's{j}_q{q}'] = float(qv[qi])
            # skew/kurtosis (manual, robust)
            xm = np.nanmean(x)
            xc = x - xm
            v = np.nanvar(xc)
            if v > 0:
                m3 = np.nanmean(xc**3)
                m4 = np.nanmean(xc**4)
                feats[f's{j}_skew'] = float(m3 / (v**1.5 + 1e-12))
                feats[f's{j}_kurt'] = float(m4 / (v**2 + 1e-12))
            else:
                feats[f's{j}_skew'] = np.nan
                feats[f's{j}_kurt'] = np.nan
            feats[f's{j}_rms'] = float(np.sqrt(np.nanmean(x*x)))
            feats[f's{j}_zcr'] = float(zero_crossings(np.nan_to_num(x))) / max(1, len(x)-1)
            a, m, c = hjorth_params(np.nan_to_num(x))
            feats[f's{j}_hj_activity'] = float(a)
            feats[f's{j}_hj_mobility'] = float(m) if np.isfinite(m) else np.nan
            feats[f's{j}_hj_complexity'] = float(c) if np.isfinite(c) else np.nan
            feats[f's{j}_trend_slope'] = float(linear_trend_slope(np.nan_to_num(x)))
            # spectral
            sf, bands = spectral_features(x, n_bands=6, downsample=10)
            for k, v in sf.items():
                feats[f's{j}_{k}'] = v
        return feats
    except Exception as e:
        return {'error': str(e)}

def build_feature_frame(file_paths, id_from_name=True):
    records = []
    names = []
    t0 = time.time()
    for idx, p in enumerate(tqdm(file_paths, desc='FE')):
        feats = extract_features_for_file(p)
        if 'error' in feats:
            print('Error on', p, feats['error'])
            continue
        records.append(feats)
        names.append(os.path.splitext(os.path.basename(p))[0])
        if (idx + 1) % 50 == 0:
            elapsed = time.time() - t0
            print(f'Processed {idx+1}/{len(file_paths)} in {elapsed:.1f}s')
    df = pd.DataFrame.from_records(records)
    if id_from_name:
        df.insert(0, 'segment_id', pd.Series(names, dtype=str))
    return df

# Subsample for smoke test
sub_n = min(200, len(train_df))
sub_ids = train_df['segment_id'].astype(str).values[:sub_n]
sub_files = [os.path.join(train_path, f'{sid}.csv') for sid in sub_ids]
print('Building features for subsample:', len(sub_files))
t0 = time.time()
train_feat_sub = build_feature_frame(sub_files)
print('Sub features shape:', train_feat_sub.shape, 'time:', f'{time.time()-t0:.1f}s')

# Merge with target for quick check
y_sub = train_df.loc[train_df['segment_id'].astype(str).isin(sub_ids), ['segment_id', 'time_to_eruption']].copy()
y_sub['segment_id'] = y_sub['segment_id'].astype(str)
train_feat_sub = train_feat_sub.merge(y_sub, on='segment_id', how='left')
print('Merged sub shape:', train_feat_sub.shape)
print(train_feat_sub.head(2).iloc[:, :12])

# Cache subsample features
out_path = 'features_sub_train.parquet'
train_feat_sub.to_parquet(out_path, index=False)
print('Wrote', out_path)

Building features for subsample: 200


FE:   0%|          | 0/200 [00:00<?, ?it/s]

/tmp/ipykernel_60/282438512.py:108: RuntimeWarning: Mean of empty slice
  feats[f's{j}_mean'] = float(np.nanmean(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/282438512.py:110: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_min'] = float(np.nanmin(x))
/tmp/ipykernel_60/282438512.py:111: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_max'] = float(np.nanmax(x))
/tmp/ipykernel_60/282438512.py:112: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_ptp'] = float(np.nanmax(x) - np.nanmin(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/tmp/ipykernel_60/282438512.py:118: RuntimeWarning: Mean of empty slice
  xm = np.nanmean(x)
/tmp/ipykernel_60/282438512.py:120: RuntimeWarning: Degrees of freedom <= 0 for 

FE:   1%|          | 2/200 [00:00<00:25,  7.76it/s]

/tmp/ipykernel_60/282438512.py:108: RuntimeWarning: Mean of empty slice
  feats[f's{j}_mean'] = float(np.nanmean(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/282438512.py:110: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_min'] = float(np.nanmin(x))
/tmp/ipykernel_60/282438512.py:111: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_max'] = float(np.nanmax(x))
/tmp/ipykernel_60/282438512.py:112: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_ptp'] = float(np.nanmax(x) - np.nanmin(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/tmp/ipykernel_60/282438512.py:118: RuntimeWarning: Mean of empty slice
  xm = np.nanmean(x)
/tmp/ipykernel_60/282438512.py:120: RuntimeWarning: Degrees of freedom <= 0 for 

...

FE:  18%|█▊        | 35/200 [00:04<00:21,  7.77it/s]

FE:  18%|█▊        | 36/200 [00:04<00:21,  7.61it/s]

FE:  18%|█▊        | 37/200 [00:04<00:21,  7.52it/s]

FE:  19%|█▉        | 38/200 [00:04<00:21,  7.54it/s]

/tmp/ipykernel_60/282438512.py:108: RuntimeWarning: Mean of empty slice
  feats[f's{j}_mean'] = float(np.nanmean(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/282438512.py:110: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_min'] = float(np.nanmin(x))
/tmp/ipykernel_60/282438512.py:111: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_max'] = float(np.nanmax(x))
/tmp/ipykernel_60/282438512.py:112: RuntimeWarning: All-NaN slice encountered
  feats[f's{j}_ptp'] = float(np.nanmax(x) - np.nanmin(x))
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1384: RuntimeWarning: All-NaN slice encountered
  return _nanquantile_unchecked(
/tmp/ipykernel_60/282438512.py:118: RuntimeWarning: Mean of empty slice
  xm = np.nanmean(x)
/tmp/ipykernel_60/282438512.py:120: RuntimeWarning: Degrees of freedom <= 0 for 

In [3]:
# Parallel Feature Extraction v1.1: robust NaN handling + joblib parallel
import os, time, math, glob, warnings
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

warnings.filterwarnings('ignore')
np.seterr(all='ignore')

train_df = pd.read_csv('train.csv')
train_dir = 'train'

def zero_crossings(x):
    x = np.asarray(x, dtype=np.float32)
    if x.size < 2:
        return 0
    return int(((x[:-1] * x[1:]) < 0).sum())

def hjorth_params(x):
    x = np.asarray(x, dtype=np.float32)
    n = x.size
    if n < 3:
        return (np.nan, np.nan, np.nan)
    dx = np.diff(x)
    ddx = np.diff(dx)
    var_x = float(np.var(x))
    var_dx = float(np.var(dx))
    var_ddx = float(np.var(ddx)) if ddx.size else np.nan
    activity = var_x
    mobility = math.sqrt(var_dx / var_x) if var_x > 0 else np.nan
    complexity = (math.sqrt(var_ddx / var_dx) / mobility) if (var_dx > 0 and mobility not in (0.0, np.nan)) else np.nan
    return activity, mobility, complexity

def linear_trend_slope(x):
    x = np.asarray(x, dtype=np.float32)
    n = x.size
    if n < 2:
        return np.nan
    idx = np.arange(n, dtype=np.float32)
    idx_mean = (n - 1) / 2.0
    x_mean = float(np.mean(x)) if n else 0.0
    cov = float(((idx - idx_mean) * (x - x_mean)).sum())
    var = float(((idx - idx_mean) ** 2).sum())
    return cov / var if var > 0 else np.nan

def spectral_features(x, n_bands=6, downsample=10):
    x = np.asarray(x, dtype=np.float32)
    if downsample > 1:
        x = x[::downsample]
    if x.size < 8:
        return {}
    x = x - float(np.mean(x))
    x = np.nan_to_num(x)
    n = x.size
    nfft = 1 << (n - 1).bit_length()
    X = np.fft.rfft(x, n=nfft)
    P = (np.abs(X) ** 2).astype(np.float32)
    if P.size == 0:
        return {}
    P[0] = 0.0
    L = len(P)
    edges = np.unique(np.round(np.geomspace(1, L - 1, num=n_bands + 1)).astype(int))
    if edges[-1] != L - 1:
        edges[-1] = L - 1
    feats = {}
    for i in range(len(edges) - 1):
        a, b = edges[i], edges[i + 1]
        if b <= a:
            feats[f'band_{i}_energy'] = 0.0
        else:
            e = float(P[a:b].sum())
            feats[f'band_{i}_energy'] = math.log1p(e)
    total = float(np.sum(P)) + 1e-12
    idx = np.arange(L, dtype=np.float32)
    feats['spec_centroid'] = float((idx * P).sum() / total)
    p = P / total
    p = p[p > 0]
    feats['spec_entropy'] = -float(np.sum(p * np.log(p))) if p.size else 0.0
    geo_mean = float(np.exp(np.mean(np.log(P[P > 0])))) if np.any(P > 0) else 0.0
    arith_mean = float(np.mean(P)) + 1e-12
    feats['spec_flatness'] = geo_mean / arith_mean
    dom_idx = int(np.argmax(P)) if P.size else 0
    feats['dom_freq_bin'] = dom_idx
    feats['dom_power_log1p'] = math.log1p(float(P[dom_idx])) if P.size else 0.0
    return feats

def per_sensor_features(x):
    # x is 1D float32, may contain NaN
    if x.size == 0:
        return {}
    feats = {}
    valid = np.isfinite(x)
    if not np.any(valid):
        # all NaN: fill informative NaNs and zeros where appropriate
        feats.update({
            'mean': np.nan, 'std': np.nan, 'min': np.nan, 'max': np.nan, 'ptp': np.nan,
            'rms': np.nan, 'zcr': 0.0, 'skew': np.nan, 'kurt': np.nan,
            'hj_activity': np.nan, 'hj_mobility': np.nan, 'hj_complexity': np.nan, 'trend_slope': np.nan
        })
        # spectral placeholders
        feats.update({
            'spec_centroid': np.nan, 'spec_entropy': np.nan, 'spec_flatness': np.nan,
            'dom_freq_bin': 0, 'dom_power_log1p': 0.0,
            **{f'band_{i}_energy': 0.0 for i in range(6)}
        })
        # quantiles
        for q in [1,5,10,25,50,75,90,95,99]:
            feats[f'q{q}'] = np.nan
        return feats
    x = x.astype(np.float32)
    x = np.where(valid, x, np.nan)
    feats['mean'] = float(np.nanmean(x))
    feats['std'] = float(np.nanstd(x))
    feats['min'] = float(np.nanmin(x))
    feats['max'] = float(np.nanmax(x))
    feats['ptp'] = feats['max'] - feats['min']
    qs = [1,5,10,25,50,75,90,95,99]
    qv = np.nanpercentile(x, qs)
    for qi, q in enumerate(qs):
        feats[f'q{q}'] = float(qv[qi])
    xm = float(np.nanmean(x))
    xc = x - xm
    v = float(np.nanvar(xc))
    if v > 0:
        m3 = float(np.nanmean(xc**3))
        m4 = float(np.nanmean(xc**4))
        feats['skew'] = m3 / (v**1.5 + 1e-12)
        feats['kurt'] = m4 / (v**2 + 1e-12)
    else:
        feats['skew'] = np.nan
        feats['kurt'] = np.nan
    feats['rms'] = float(np.sqrt(np.nanmean(x*x)))
    feats['zcr'] = float(zero_crossings(np.nan_to_num(x))) / max(1, x.size - 1)
    a, m, c = hjorth_params(np.nan_to_num(x))
    feats['hj_activity'] = float(a)
    feats['hj_mobility'] = float(m) if np.isfinite(m) else np.nan
    feats['hj_complexity'] = float(c) if np.isfinite(c) else np.nan
    feats['trend_slope'] = float(linear_trend_slope(np.nan_to_num(x)))
    # spectral
    feats.update(spectral_features(x, n_bands=6, downsample=10))
    return feats

def extract_one(path):
    seg_id = os.path.splitext(os.path.basename(path))[0]
    try:
        df = pd.read_csv(path)
        arr = df.values.astype(np.float32)
        T, C = arr.shape
        out = {'segment_id': seg_id, 'rows': T, 'cols': C, 'missing_frac': float(np.isnan(arr).mean())}
        for j in range(C):
            f = per_sensor_features(arr[:, j])
            for k, v in f.items():
                out[f's{j}_{k}'] = v
        return out
    except Exception as e:
        return {'segment_id': seg_id, 'error': str(e)}

# Fast smoke parallel run on 80 segments
sub_n = min(80, len(train_df))
sub_ids = train_df['segment_id'].astype(str).values[:sub_n]
sub_files = [os.path.join(train_dir, f'{sid}.csv') for sid in sub_ids]
print('Parallel FE on subsample:', len(sub_files))
t0 = time.time()
n_jobs = max(1, os.cpu_count() - 2)
results = Parallel(n_jobs=n_jobs, prefer='processes', batch_size=2)(delayed(extract_one)(p) for p in sub_files)
elapsed = time.time() - t0
print(f'FE done in {elapsed:.1f}s with n_jobs={n_jobs}')
err = [r for r in results if 'error' in r]
print('Errors:', len(err))
feat_df = pd.DataFrame([r for r in results if 'error' not in r])
feat_df = feat_df.merge(train_df[['segment_id', 'time_to_eruption']].astype({'segment_id': str}), on='segment_id', how='left')
print('Sub features shape:', feat_df.shape)
print('Columns:', len(feat_df.columns))
feat_df.to_parquet('features_sub_train_parallel.parquet', index=False)
print('Wrote features_sub_train_parallel.parquet')

Parallel FE on subsample: 80


FE done in 2.0s with n_jobs=34
Errors: 0
Sub features shape: (80, 335)
Columns: 335
Wrote features_sub_train_parallel.parquet


In [4]:
# Full Feature Extraction (train/test) with parallelism and grouping aids
import os, time, math, warnings
import numpy as np
import pandas as pd
from joblib import Parallel, delayed

warnings.filterwarnings('ignore')
np.seterr(all='ignore')

train_df = pd.read_csv('train.csv')
ss_df = pd.read_csv('sample_submission.csv')
train_dir, test_dir = 'train', 'test'

def zero_crossings(x):
    x = np.asarray(x, dtype=np.float32)
    if x.size < 2:
        return 0
    return int(((x[:-1] * x[1:]) < 0).sum())

def hjorth_params(x):
    x = np.asarray(x, dtype=np.float32)
    n = x.size
    if n < 3:
        return (np.nan, np.nan, np.nan)
    dx = np.diff(x)
    ddx = np.diff(dx)
    var_x = float(np.var(x))
    var_dx = float(np.var(dx))
    var_ddx = float(np.var(ddx)) if ddx.size else np.nan
    activity = var_x
    mobility = math.sqrt(var_dx / var_x) if var_x > 0 else np.nan
    complexity = (math.sqrt(var_ddx / var_dx) / mobility) if (var_dx > 0 and mobility not in (0.0, np.nan)) else np.nan
    return activity, mobility, complexity

def linear_trend_slope(x):
    x = np.asarray(x, dtype=np.float32)
    n = x.size
    if n < 2:
        return np.nan
    idx = np.arange(n, dtype=np.float32)
    idx_mean = (n - 1) / 2.0
    x_mean = float(np.mean(x)) if n else 0.0
    cov = float(((idx - idx_mean) * (x - x_mean)).sum())
    var = float(((idx - idx_mean) ** 2).sum())
    return cov / var if var > 0 else np.nan

def spectral_features(x, n_bands=6, downsample=10):
    x = np.asarray(x, dtype=np.float32)
    if downsample > 1:
        x = x[::downsample]
    if x.size < 8:
        return {}
    x = x - float(np.mean(x))
    x = np.nan_to_num(x)
    n = x.size
    nfft = 1 << (n - 1).bit_length()
    X = np.fft.rfft(x, n=nfft)
    P = (np.abs(X) ** 2).astype(np.float32)
    if P.size == 0:
        return {}
    P[0] = 0.0
    L = len(P)
    edges = np.unique(np.round(np.geomspace(1, L - 1, num=n_bands + 1)).astype(int))
    if edges[-1] != L - 1:
        edges[-1] = L - 1
    feats = {}
    for i in range(len(edges) - 1):
        a, b = edges[i], edges[i + 1]
        if b <= a:
            feats[f'band_{i}_energy'] = 0.0
        else:
            e = float(P[a:b].sum())
            feats[f'band_{i}_energy'] = math.log1p(e)
    total = float(np.sum(P)) + 1e-12
    idx = np.arange(L, dtype=np.float32)
    feats['spec_centroid'] = float((idx * P).sum() / total)
    p = P / total
    p = p[p > 0]
    feats['spec_entropy'] = -float(np.sum(p * np.log(p))) if p.size else 0.0
    geo_mean = float(np.exp(np.mean(np.log(P[P > 0])))) if np.any(P > 0) else 0.0
    arith_mean = float(np.mean(P)) + 1e-12
    feats['spec_flatness'] = geo_mean / arith_mean
    dom_idx = int(np.argmax(P)) if P.size else 0
    feats['dom_freq_bin'] = dom_idx
    feats['dom_power_log1p'] = math.log1p(float(P[dom_idx])) if P.size else 0.0
    return feats

def per_sensor_features(x):
    feats = {}
    if x.size == 0:
        return feats
    valid = np.isfinite(x)
    feats['all_nan'] = float(1.0 if not np.any(valid) else 0.0)
    if not np.any(valid):
        base = {
            'mean': np.nan, 'std': np.nan, 'min': np.nan, 'max': np.nan, 'ptp': np.nan,
            'rms': np.nan, 'zcr': 0.0, 'skew': np.nan, 'kurt': np.nan,
            'hj_activity': np.nan, 'hj_mobility': np.nan, 'hj_complexity': np.nan, 'trend_slope': np.nan,
            'spec_centroid': np.nan, 'spec_entropy': np.nan, 'spec_flatness': np.nan,
            'dom_freq_bin': 0, 'dom_power_log1p': 0.0
        }
        base.update({f'band_{i}_energy': 0.0 for i in range(6)})
        for q in [1,5,10,25,50,75,90,95,99]:
            base[f'q{q}'] = np.nan
        feats.update(base)
        return feats
    x = x.astype(np.float32)
    x = np.where(valid, x, np.nan)
    feats['mean'] = float(np.nanmean(x))
    feats['std'] = float(np.nanstd(x))
    feats['min'] = float(np.nanmin(x))
    feats['max'] = float(np.nanmax(x))
    feats['ptp'] = feats['max'] - feats['min']
    qs = [1,5,10,25,50,75,90,95,99]
    qv = np.nanpercentile(x, qs)
    for qi, q in enumerate(qs):
        feats[f'q{q}'] = float(qv[qi])
    xm = float(np.nanmean(x))
    xc = x - xm
    v = float(np.nanvar(xc))
    if v > 0:
        m3 = float(np.nanmean(xc**3))
        m4 = float(np.nanmean(xc**4))
        feats['skew'] = m3 / (v**1.5 + 1e-12)
        feats['kurt'] = m4 / (v**2 + 1e-12)
    else:
        feats['skew'] = np.nan
        feats['kurt'] = np.nan
    feats['rms'] = float(np.sqrt(np.nanmean(x*x)))
    feats['zcr'] = float(zero_crossings(np.nan_to_num(x))) / max(1, x.size - 1)
    a, m, c = hjorth_params(np.nan_to_num(x))
    feats['hj_activity'] = float(a)
    feats['hj_mobility'] = float(m) if np.isfinite(m) else np.nan
    feats['hj_complexity'] = float(c) if np.isfinite(c) else np.nan
    feats['trend_slope'] = float(linear_trend_slope(np.nan_to_num(x)))
    feats.update(spectral_features(x, n_bands=6, downsample=10))
    return feats

def extract_one(path):
    seg_id = os.path.splitext(os.path.basename(path))[0]
    try:
        df = pd.read_csv(path)
        arr = df.values.astype(np.float32)
        T, C = arr.shape
        out = {'segment_id': seg_id, 'rows': T, 'cols': C}
        nan_mask = np.isnan(arr)
        out['missing_frac'] = float(nan_mask.mean())
        out['active_sensors'] = float((~np.all(nan_mask, axis=0)).sum())
        for j in range(C):
            f = per_sensor_features(arr[:, j])
            for k, v in f.items():
                out[f's{j}_{k}'] = v
        return out
    except Exception as e:
        return {'segment_id': seg_id, 'error': str(e)}

def run_parallel(file_ids, data_dir, out_path, n_jobs=12):
    files = [os.path.join(data_dir, f'{sid}.csv') for sid in file_ids]
    t0 = time.time()
    results = Parallel(n_jobs=n_jobs, prefer='processes', batch_size=2)(delayed(extract_one)(p) for p in files)
    elapsed = time.time() - t0
    print(f'Extracted {len(files)} segments in {elapsed:.1f}s (n_jobs={n_jobs})')
    errs = [r for r in results if 'error' in r]
    if errs:
        print('Errors:', len(errs), 'Example:', errs[0])
    feat_df = pd.DataFrame([r for r in results if 'error' not in r])
    feat_df.to_parquet(out_path, index=False)
    print('Wrote', out_path, 'shape:', feat_df.shape)
    return feat_df

# Execute full extraction
train_ids = train_df['segment_id'].astype(str).tolist()
test_ids = ss_df['segment_id'].astype(str).tolist()
n_jobs = min(16, max(2, os.cpu_count() // 2))
print('Using n_jobs=', n_jobs)
train_feats_path = 'features_train.parquet'
test_feats_path = 'features_test.parquet'

if not os.path.exists(train_feats_path):
    _ = run_parallel(train_ids, train_dir, train_feats_path, n_jobs=n_jobs)
else:
    print('Found cached', train_feats_path)
if not os.path.exists(test_feats_path):
    _ = run_parallel(test_ids, test_dir, test_feats_path, n_jobs=n_jobs)
else:
    print('Found cached', test_feats_path)

Using n_jobs= 16


Extracted 3987 segments in 40.0s (n_jobs=16)


Wrote features_train.parquet shape: (3987, 345)


Extracted 444 segments in 4.3s (n_jobs=16)
Wrote features_test.parquet shape: (444, 345)


In [5]:
# GroupKFold + XGBoost GPU baseline, OOF MAE, and submission
import os, time, re
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

train_df = pd.read_parquet('features_train.parquet')
test_df = pd.read_parquet('features_test.parquet')
y_df = pd.read_csv('train.csv').astype({'segment_id': str})
ss = pd.read_csv('sample_submission.csv').astype({'segment_id': str})

# Merge target
train_df = train_df.merge(y_df, on='segment_id', how='left')

# Align feature columns between train/test: union, fill missing with NaN
ignore_cols = {'segment_id', 'time_to_eruption'}
train_feats = [c for c in train_df.columns if c not in ignore_cols]
test_feats = [c for c in test_df.columns if c not in {'segment_id'}]
all_feats = sorted(set(train_feats).union(test_feats))
for c in all_feats:
    if c not in train_df.columns:
        train_df[c] = np.nan
    if c not in test_df.columns:
        test_df[c] = np.nan

# Derive clustering features per expert advice
def make_cluster_frame(df):
    cols = []
    # all_nan flags
    cols += [c for c in df.columns if re.fullmatch(r's\d+_all_nan', c) is not None]
    basic = ['active_sensors', 'missing_frac']
    cols += [c for c in basic if c in df.columns]
    # aggregated spectral shape: per-sensor spec_centroid, dom_freq_bin, low-band energy band_0_energy
    spec_cent_cols = [c for c in df.columns if re.fullmatch(r's\d+_spec_centroid', c) is not None]
    dom_bin_cols = [c for c in df.columns if re.fullmatch(r's\d+_dom_freq_bin', c) is not None]
    band0_cols = [c for c in df.columns if re.fullmatch(r's\d+_band_0_energy', c) is not None]
    agg = pd.DataFrame(index=df.index)
    for name, lst in [('spec_centroid', spec_cent_cols), ('dom_freq_bin', dom_bin_cols), ('band0', band0_cols)]:
        if lst:
            vals = df[lst].astype(float)
            agg[f'{name}_mean'] = vals.mean(axis=1)
            agg[f'{name}_std'] = vals.std(axis=1)
    base = df[cols].copy() if cols else pd.DataFrame(index=df.index)
    cdf = pd.concat([base, agg], axis=1)
    return cdf

cluster_train = make_cluster_frame(train_df)
cluster_test = make_cluster_frame(test_df)
# Fill NaNs with 0 for clustering and scale
scaler = StandardScaler()
Xc = scaler.fit_transform(cluster_train.fillna(0.0)) if cluster_train.shape[1] > 0 else np.zeros((len(train_df), 1))
Xt = scaler.transform(cluster_test.fillna(0.0)) if cluster_test.shape[1] > 0 else np.zeros((len(test_df), 1))

# KMeans clustering
k = 6
km = KMeans(n_clusters=k, random_state=42, n_init=10)
groups = km.fit_predict(Xc) if Xc.shape[1] > 0 else np.zeros(len(train_df), dtype=int)
train_df['cluster'] = groups
print('Cluster sizes:', pd.Series(groups).value_counts().to_dict())

# Prepare matrices
feature_cols = [c for c in all_feats]
X = train_df[feature_cols].astype(np.float32).values
y = train_df['time_to_eruption'].values.astype(np.float32)
X_test = test_df[feature_cols].astype(np.float32).values
groups_arr = train_df['cluster'].values

# 5-fold GroupKFold
gkf = GroupKFold(n_splits=5)
oof = np.zeros(len(train_df), dtype=np.float32)
test_pred_folds = []
fold_maes = []

params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'objective': 'reg:absoluteerror',
    'eval_metric': 'mae',
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.75,
    'min_child_weight': 6,
    'reg_alpha': 1.0,
    'reg_lambda': 8.0,
    'max_bin': 256,
    'random_state': 42
}

n_estimators = 5000
early_stopping_rounds = 200

t_total = time.time()
for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, y, groups=groups_arr), 1):
    t0 = time.time()
    X_tr, X_va = X[tr_idx], X[va_idx]
    y_tr, y_va = y[tr_idx], y[va_idx]
    dtrain = xgb.DMatrix(X_tr, label=y_tr, nthread=-1)
    dvalid = xgb.DMatrix(X_va, label=y_va, nthread=-1)
    dtest = xgb.DMatrix(X_test, nthread=-1)
    evals = [(dtrain, 'train'), (dvalid, 'valid')]
    model = xgb.train(params, dtrain, num_boost_round=n_estimators, evals=evals, early_stopping_rounds=early_stopping_rounds, verbose_eval=200)
    oof[va_idx] = model.predict(dvalid, iteration_range=(0, model.best_iteration + 1))
    fold_test = model.predict(dtest, iteration_range=(0, model.best_iteration + 1))
    test_pred_folds.append(fold_test)
    mae = mean_absolute_error(y_va, oof[va_idx])
    fold_maes.append(mae)
    print(f'Fold {fold} MAE: {mae:.2f} | best_iter {model.best_iteration} | elapsed {time.time()-t0:.1f}s', flush=True)

oof_mae = mean_absolute_error(y, oof)
print('OOF MAE:', oof_mae, 'folds:', [round(m,2) for m in fold_maes])

# Test prediction: average over folds
test_pred = np.mean(np.vstack(test_pred_folds), axis=0).astype(np.float32)

# Clip predictions
ymax = float(y.max())
clip_max = ymax * 1.05
oof_clipped = np.clip(oof, 0.0, clip_max)
test_pred_clipped = np.clip(test_pred, 0.0, clip_max)
oof_mae_clipped = mean_absolute_error(y, oof_clipped)
print('OOF MAE (clipped):', oof_mae_clipped)

# Optional bias correction: global median residual
resid = y - oof_clipped
bias = float(np.median(resid))
print('Median residual (bias):', bias)
test_pred_bc = np.clip(test_pred_clipped + bias, 0.0, clip_max)

# Build submission
sub = ss.copy()
sub['time_to_eruption'] = test_pred_bc.astype(np.int64)
sub_path = 'submission.csv'
sub.to_csv(sub_path, index=False)
print('Saved submission to', sub_path, 'shape:', sub.shape)

Cluster sizes: {1: 1581, 5: 1309, 0: 509, 2: 451, 3: 115, 4: 22}


[0]	train-mae:10971203.19742	valid-mae:11912628.62176


[200]	train-mae:915897.79729	valid-mae:8060441.59709


[400]	train-mae:606875.64876	valid-mae:7886861.18042


[600]	train-mae:507404.00388	valid-mae:7827702.95754


...

Fold 4 MAE: 11861471.00 | best_iter 11 | elapsed 3.3s


[0]	train-mae:11229989.26286	valid-mae:9296863.29927


[200]	train-mae:1069407.31072	valid-mae:12864830.50365


Fold 5 MAE: 9296863.00 | best_iter 0 | elapsed 3.2s


OOF MAE: 8426957.0 folds: [7778755.0, 6813938.5, 11311244.0, 11861471.0, 9296863.0]
OOF MAE (clipped): 8426957.0
Median residual (bias): 16996.0
Saved submission to submission.csv shape: (444, 2)


In [6]:
# Feature Extraction v2: Add Welch PSD bands + Hilbert envelope + 3-window stats; cache to v2 parquet
import os, time, math, warnings
import numpy as np
import pandas as pd
from joblib import Parallel, delayed
from scipy.signal import welch, hilbert

warnings.filterwarnings('ignore')
np.seterr(all='ignore')

train_ids = pd.read_csv('train.csv')['segment_id'].astype(str).tolist()
test_ids = pd.read_csv('sample_submission.csv')['segment_id'].astype(str).tolist()
train_dir, test_dir = 'train', 'test'

def three_window_stats(x):
    n = x.size
    if n < 3:
        return {}
    thirds = n // 3
    a = x[:thirds]
    b = x[thirds:2*thirds]
    c = x[2*thirds:]
    def mstats(z):
        return float(np.nanmean(z)), float(np.nanstd(z))
    ma, sa = mstats(a)
    mb, sb = mstats(b)
    mc, sc = mstats(c)
    return {
        'w_start_mean': ma, 'w_start_std': sa,
        'w_mid_mean': mb, 'w_mid_std': sb,
        'w_end_mean': mc, 'w_end_std': sc,
        'w_delta_mean_end_start': mc - ma
    }

def welch_psd_feats(x, fs=100.0, nperseg=512, noverlap=256, n_bands=8):
    # If true fs unknown, treat index as time with fs=100 for stable banding
    x = np.nan_to_num(x.astype(np.float32))
    if x.size < nperseg:
        # fallback: rfft bands like v1
        return {}
    f, Pxx = welch(x, fs=fs, nperseg=nperseg, noverlap=noverlap, detrend='constant', scaling='density')
    if Pxx.size == 0:
        return {}
    # Log-spaced bands up to Nyquist
    fmin, fmax = f[1], f[-1]
    edges = np.unique(np.round(np.geomspace(max(1e-3, fmin), fmax, num=n_bands + 1), 5))
    feats = {}
    total = float(np.trapz(Pxx, f)) + 1e-12
    for i in range(len(edges) - 1):
        a, b = edges[i], edges[i+1]
        mask = (f >= a) & (f < b)
        e = float(np.trapz(Pxx[mask], f[mask])) if np.any(mask) else 0.0
        feats[f'welch_band_{i}_loge'] = math.log1p(e)
    # Spectral summary
    p = Pxx / (np.sum(Pxx) + 1e-12)
    p_pos = p[p > 0]
    feats['welch_entropy'] = -float(np.sum(p_pos * np.log(p_pos))) if p_pos.size else 0.0
    geo = float(np.exp(np.mean(np.log(Pxx[Pxx > 0])))) if np.any(Pxx > 0) else 0.0
    ari = float(np.mean(Pxx)) + 1e-12
    feats['welch_flatness'] = geo / ari
    # rolloffs
    cumsum = np.cumsum(Pxx) / (np.sum(Pxx) + 1e-12)
    def rolloff(th):
        idx = np.searchsorted(cumsum, th)
        idx = np.clip(idx, 0, len(f)-1)
        return float(f[idx])
    feats['welch_rolloff85'] = rolloff(0.85)
    feats['welch_rolloff95'] = rolloff(0.95)
    return feats

def hilbert_envelope_feats(x):
    x = np.nan_to_num(x.astype(np.float32))
    if x.size < 8:
        return {}
    env = np.abs(hilbert(x))
    qs = [10,25,50,75,90]
    qv = np.percentile(env, qs)
    feats = {
        'hilb_mean': float(env.mean()),
        'hilb_std': float(env.std()),
        'hilb_q10': float(qv[0]), 'hilb_q25': float(qv[1]), 'hilb_q50': float(qv[2]),
        'hilb_q75': float(qv[3]), 'hilb_q90': float(qv[4])
    }
    return feats

def extract_v2_one(path):
    seg_id = os.path.splitext(os.path.basename(path))[0]
    try:
        df = pd.read_csv(path)
        arr = df.values.astype(np.float32)  # (T, C)
        T, C = arr.shape
        out = {'segment_id': seg_id}
        # choose a nominal fs (unknown in data); relative features still useful
        fs = 100.0
        for j in range(C):
            x = arr[:, j]
            # Welch PSD features
            wfe = welch_psd_feats(x, fs=fs, nperseg=512, noverlap=256, n_bands=8)
            for k, v in wfe.items():
                out[f's{j}_{k}'] = v
            # Hilbert envelope features
            hfe = hilbert_envelope_feats(x)
            for k, v in hfe.items():
                out[f's{j}_{k}'] = v
            # 3-window stats
            tw = three_window_stats(x)
            for k, v in tw.items():
                out[f's{j}_{k}'] = v
        return out
    except Exception as e:
        return {'segment_id': seg_id, 'error': str(e)}

def run_parallel_v2(ids, data_dir, out_path, n_jobs=16):
    files = [os.path.join(data_dir, f'{sid}.csv') for sid in ids]
    t0 = time.time()
    res = Parallel(n_jobs=n_jobs, prefer='processes', batch_size=2)(delayed(extract_v2_one)(p) for p in files)
    dt = time.time() - t0
    errs = [r for r in res if 'error' in r]
    if errs:
        print('Errors:', len(errs), 'Example:', errs[0])
    df = pd.DataFrame([r for r in res if 'error' not in r])
    df.to_parquet(out_path, index=False)
    print(f'Wrote {out_path} shape {df.shape} in {dt:.1f}s (n_jobs={n_jobs})')
    return df

n_jobs = min(24, max(4, os.cpu_count() // 2))
print('FE v2 using n_jobs=', n_jobs)
train_v2 = 'features_train_v2.parquet'
test_v2 = 'features_test_v2.parquet'
if not os.path.exists(train_v2):
    _ = run_parallel_v2(train_ids, train_dir, train_v2, n_jobs=n_jobs)
else:
    print('Found cached', train_v2)
if not os.path.exists(test_v2):
    _ = run_parallel_v2(test_ids, test_dir, test_v2, n_jobs=n_jobs)
else:
    print('Found cached', test_v2)

# Merge v2 features into existing v1 caches for modeling
base_tr = pd.read_parquet('features_train.parquet')
base_te = pd.read_parquet('features_test.parquet')
v2_tr = pd.read_parquet(train_v2)
v2_te = pd.read_parquet(test_v2)
full_tr = base_tr.merge(v2_tr, on='segment_id', how='left')
full_te = base_te.merge(v2_te, on='segment_id', how='left')
full_tr.to_parquet('features_train_full.parquet', index=False)
full_te.to_parquet('features_test_full.parquet', index=False)
print('Wrote features_train_full.parquet', full_tr.shape)
print('Wrote features_test_full.parquet', full_te.shape)

FE v2 using n_jobs= 18


/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

...

/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/tmp/ipykernel_60/294650540.py:24: RuntimeWarning: Mean of empty slice
/usr/local/lib/python3.11/dist-packages/numpy/lib/nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dty

Wrote features_test_v2.parquet shape (444, 261) in 6.4s (n_jobs=18)


Wrote features_train_full.parquet (3987, 605)
Wrote features_test_full.parquet (444, 605)


In [7]:
# Train XGBoost on full features (v1+v2), improved clustering (merge tiny clusters)
import os, re, time
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

tr_path = 'features_train_full.parquet'
te_path = 'features_test_full.parquet'
assert os.path.exists(tr_path) and os.path.exists(te_path), 'Full features not found'
train_df = pd.read_parquet(tr_path)
test_df = pd.read_parquet(te_path)
y_df = pd.read_csv('train.csv').astype({'segment_id': str})
ss = pd.read_csv('sample_submission.csv').astype({'segment_id': str})
train_df = train_df.merge(y_df, on='segment_id', how='left')

# Align columns
ignore_cols = {'segment_id', 'time_to_eruption'}
tr_feats = [c for c in train_df.columns if c not in ignore_cols]
te_feats = [c for c in test_df.columns if c != 'segment_id']
all_feats = sorted(set(tr_feats).union(te_feats))
for c in all_feats:
    if c not in train_df.columns: train_df[c] = np.nan
    if c not in test_df.columns: test_df[c] = np.nan

# Build clustering features (missingness + simple spectral aggregates incl. Welch low band)
def make_cluster_frame(df):
    parts = []
    # all_nan flags + basic missingness
    all_nan_cols = [c for c in df.columns if re.fullmatch(r's\d+_all_nan', c) is not None]
    basic = [c for c in ['active_sensors','missing_frac'] if c in df.columns]
    if all_nan_cols or basic:
        parts.append(df[all_nan_cols + basic].astype(float))
    # aggregate per-sensor spectral markers (from v1 and v2)
    agg = pd.DataFrame(index=df.index)
    def add_agg(prefix, cols):
        if cols:
            vals = df[cols].astype(float)
            agg[f'{prefix}_mean'] = vals.mean(axis=1)
            agg[f'{prefix}_std'] = vals.std(axis=1)
    add_agg('spec_centroid', [c for c in df.columns if re.fullmatch(r's\d+_spec_centroid', c)])
    add_agg('dom_freq_bin', [c for c in df.columns if re.fullmatch(r's\d+_dom_freq_bin', c)])
    add_agg('band0', [c for c in df.columns if re.fullmatch(r's\d+_band_0_energy', c)])
    add_agg('welch_band0', [c for c in df.columns if re.fullmatch(r's\d+_welch_band_0_loge', c)])
    if not agg.empty:
        parts.append(agg)
    return pd.concat(parts, axis=1) if parts else pd.DataFrame(index=df.index)

cluster_train = make_cluster_frame(train_df)
cluster_test = make_cluster_frame(test_df)
cluster_train = cluster_train.fillna(0.0)
cluster_test = cluster_test.fillna(0.0)
scaler = StandardScaler()
Xc_tr = scaler.fit_transform(cluster_train) if cluster_train.shape[1] > 0 else np.zeros((len(train_df),1))
Xc_te = scaler.transform(cluster_test) if cluster_test.shape[1] > 0 else np.zeros((len(test_df),1))

# KMeans and merge tiny clusters (<100) into nearest
k = 6
km = KMeans(n_clusters=k, random_state=42, n_init=10)
labels = km.fit_predict(Xc_tr) if Xc_tr.shape[1] > 0 else np.zeros(len(train_df), dtype=int)
train_df['cluster'] = labels
sizes = pd.Series(labels).value_counts().sort_index()
print('Initial cluster sizes:', sizes.to_dict())
tiny = sizes[sizes < 100].index.tolist()
if len(tiny) > 0:
    centers = km.cluster_centers_
    for t in tiny:
        # find nearest non-tiny cluster center
        d = np.linalg.norm(centers - centers[t], axis=1)
        d[t] = np.inf
        # restrict to non-tiny
        for tid in tiny:
            d[tid] = np.inf
        nearest = int(np.argmin(d)) if np.isfinite(d).any() else int(np.argmax(sizes.values))
        train_df.loc[train_df['cluster']==t, 'cluster'] = nearest
    sizes2 = train_df['cluster'].value_counts().sort_index()
    print('Merged cluster sizes:', sizes2.to_dict())

# Prepare matrices
X = train_df[all_feats].astype(np.float32).values
y = train_df['time_to_eruption'].astype(np.float32).values
X_test = test_df[all_feats].astype(np.float32).values
groups = train_df['cluster'].values

# XGBoost params
params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'objective': 'reg:absoluteerror',
    'eval_metric': 'mae',
    'max_depth': 8,
    'learning_rate': 0.05,
    'subsample': 0.8,
    'colsample_bytree': 0.75,
    'min_child_weight': 6,
    'reg_alpha': 1.0,
    'reg_lambda': 8.0,
    'max_bin': 256,
    'random_state': 42
}

gkf = GroupKFold(n_splits=5)
oof = np.zeros(len(train_df), dtype=np.float32)
test_preds = []
fold_maes = []
n_estimators = 5000
esr = 200

for fold, (tr_idx, va_idx) in enumerate(gkf.split(X, y, groups=groups), 1):
    t0 = time.time()
    dtr = xgb.DMatrix(X[tr_idx], label=y[tr_idx], nthread=-1)
    dva = xgb.DMatrix(X[va_idx], label=y[va_idx], nthread=-1)
    dte = xgb.DMatrix(X_test, nthread=-1)
    model = xgb.train(params, dtr, num_boost_round=n_estimators, evals=[(dtr,'train'),(dva,'valid')], early_stopping_rounds=esr, verbose_eval=200)
    oof[va_idx] = model.predict(dva, iteration_range=(0, model.best_iteration+1))
    test_preds.append(model.predict(dte, iteration_range=(0, model.best_iteration+1)))
    mae = mean_absolute_error(y[va_idx], oof[va_idx])
    fold_maes.append(mae)
    print(f'Fold {fold} MAE: {mae:.2f} | best_iter {model.best_iteration} | {time.time()-t0:.1f}s', flush=True)

oof_mae = mean_absolute_error(y, oof)
print('OOF MAE(full features):', oof_mae, 'folds:', [round(m,2) for m in fold_maes])

# Test prediction
pred = np.mean(np.vstack(test_preds), axis=0).astype(np.float32)
clip_max = float(y.max()) * 1.05
pred = np.clip(pred, 0.0, clip_max)
bias = float(np.median(y - np.clip(oof, 0.0, clip_max)))
pred_bc = np.clip(pred + bias, 0.0, clip_max)

sub = ss.copy()
sub['time_to_eruption'] = pred_bc.astype(np.int64)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv with full features. Shape:', sub.shape)

Initial cluster sizes: {0: 434, 1: 1065, 2: 1421, 3: 922, 4: 123, 5: 22}
Merged cluster sizes: {0: 434, 1: 1065, 2: 1421, 3: 944, 4: 123}
[0]	train-mae:11242151.21863	valid-mae:11447252.49120


[200]	train-mae:911666.45982	valid-mae:8299827.33361


[400]	train-mae:590160.82834	valid-mae:8064907.51060


[600]	train-mae:491018.01585	valid-mae:7983024.41692


[800]	train-mae:432194.03156	valid-mae:7948406.08251


...

[0]	train-mae:11241264.55435	valid-mae:7669970.55285


[200]	train-mae:970306.04678	valid-mae:11609919.31707


[207]	train-mae:949546.20540	valid-mae:11603845.39837


Fold 5 MAE: 7581469.50 | best_iter 7 | 5.0s


OOF MAE(full features): 8991907.0 folds: [7597515.0, 7798913.5, 11245940.0, 11981876.0, 7581469.5]
Saved submission.csv with full features. Shape: (444, 2)


In [ ]:
# Stable CV: StratifiedKFold on y-bins + XGBoost (more regularized) using full features
import numpy as np, pandas as pd, time
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_absolute_error
import xgboost as xgb

tr_path, te_path = 'features_train_full.parquet', 'features_test_full.parquet'
train_df = pd.read_parquet(tr_path)
test_df = pd.read_parquet(te_path)
y_df = pd.read_csv('train.csv').astype({'segment_id': str})
ss = pd.read_csv('sample_submission.csv').astype({'segment_id': str})
train_df = train_df.merge(y_df, on='segment_id', how='left')

# Align columns
ignore_cols = {'segment_id', 'time_to_eruption'}
tr_feats = [c for c in train_df.columns if c not in ignore_cols]
te_feats = [c for c in test_df.columns if c != 'segment_id']
all_feats = sorted(set(tr_feats).union(te_feats))
for c in all_feats:
    if c not in train_df.columns: train_df[c] = np.nan
    if c not in test_df.columns: test_df[c] = np.nan

X = train_df[all_feats].astype(np.float32).values
y = train_df['time_to_eruption'].astype(np.float32).values
X_test = test_df[all_feats].astype(np.float32).values

# Create y bins for stratification
bins = pd.qcut(y, q=10, labels=False, duplicates='drop')

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
oof = np.zeros(len(train_df), dtype=np.float32)
test_preds = []
fold_maes = []

params = {
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'objective': 'reg:squarederror',  # more stable; still eval MAE
    'eval_metric': 'mae',
    'max_depth': 6,
    'learning_rate': 0.03,
    'subsample': 0.75,
    'colsample_bytree': 0.7,
    'min_child_weight': 10,
    'reg_alpha': 2.0,
    'reg_lambda': 12.0,
    'gamma': 0.2,
    'max_bin': 256,
    'random_state': 42
}

n_estimators = 8000
esr = 200

for fold, (tr_idx, va_idx) in enumerate(skf.split(X, bins), 1):
    t0 = time.time()
    y_tr, y_va = y[tr_idx], y[va_idx]
    print(f'Fold {fold} stats: y_tr mean {y_tr.mean():.1f} std {y_tr.std():.1f} | y_va mean {y_va.mean():.1f} std {y_va.std():.1f}', flush=True)
    dtr = xgb.DMatrix(X[tr_idx], label=y_tr, nthread=-1)
    dva = xgb.DMatrix(X[va_idx], label=y_va, nthread=-1)
    dte = xgb.DMatrix(X_test, nthread=-1)
    model = xgb.train(params, dtr, num_boost_round=n_estimators, evals=[(dtr,'train'),(dva,'valid')], early_stopping_rounds=esr, verbose_eval=250)
    oof[va_idx] = model.predict(dva, iteration_range=(0, model.best_iteration+1))
    test_preds.append(model.predict(dte, iteration_range=(0, model.best_iteration+1)))
    mae = mean_absolute_error(y_va, oof[va_idx])
    fold_maes.append(mae)
    print(f'Fold {fold} MAE: {mae:.2f} | best_iter {model.best_iteration} | {time.time()-t0:.1f}s', flush=True)

oof_mae = mean_absolute_error(y, oof)
clip_max = float(y.max()) * 1.05
oof_clip = np.clip(oof, 0.0, clip_max)
oof_mae_clip = mean_absolute_error(y, oof_clip)
print('OOF MAE (StratKFold):', oof_mae, 'clipped:', oof_mae_clip, 'folds:', [round(m,2) for m in fold_maes])

pred = np.mean(np.vstack(test_preds), axis=0).astype(np.float32)
pred = np.clip(pred, 0.0, clip_max)
bias = float(np.median(y - oof_clip))
pred_bc = np.clip(pred + bias, 0.0, clip_max)

sub = ss.copy()
sub['time_to_eruption'] = pred_bc.astype(np.int64)
sub.to_csv('submission.csv', index=False)
print('Saved submission.csv (StratKFold). Shape:', sub.shape)

Fold 1 stats: y_tr mean 22681942.0 std 13506437.0 | y_va mean 22651894.0 std 13416435.0


[0]	train-mae:11428520.63217	valid-mae:11370063.03008


[250]	train-mae:1256824.35677	valid-mae:2374721.42513


[500]	train-mae:580333.78866	valid-mae:2129276.58850
